In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

rootpath = '/media/share/data/kaggle/ieee-camera/'

# Preprocess

In [11]:
def df_generater(rootpath, dataset):
    # find all the camera
    subdir = os.path.join(rootpath, dataset)
    n_class = os.listdir(subdir)
    df_temp = []

    for i in n_class:
        for fname in os.listdir(os.path.join(subdir, i)):
            df_temp.append((i, fname))

    df = pd.DataFrame(df_temp, columns=['class', 'fname'])

    return df, n_class


def image_crop(indexlist):
    df, n_class = df_generater(rootpath, 'train')
    subpath = os.path.join(rootpath, 'train')
    
    crop_size=512
    img_file = []
    label = []
    for idx in indexlist:
        img = plt.imread(os.path.join(subpath, df['class'][idx]) + '/' + df['fname'][idx])

        h1, w1, _ = img.shape
        h2 = int(h1/int(h1/crop_size))
        w2 = int(w1/int(w1/crop_size))
        
        for i in range(int(h1/crop_size)):
            for j in range(int(w1/crop_size)):
                hr = np.random.randint(i*h2, (i+1)*h2-crop_size)
                wr = np.random.randint(j*w2, (j+1)*w2-crop_size)
                tem_img = img[hr:hr+crop_size, wr:wr+crop_size, :]
                img_file.append(tem_img)
                label.append(df['class'][idx])
                
    from sklearn.preprocessing import LabelEncoder
    from keras.utils import to_categorical
    le = LabelEncoder().fit(n_class)
    labels = to_categorical(le.transform(label), num_classes=len(n_class))
    
    return np.array(img_file), labels

In [3]:
from sklearn.model_selection import StratifiedKFold

df_train, n_class = df_generater(rootpath, dataset='train')
n_fold = 5
skf = StratifiedKFold(n_fold, shuffle=True, random_state=np.random)
for train_idx, val_idx in skf.split(df_train['fname'], df_train['class']):
    break

# CNN model

In [6]:
import keras.backend as K
from keras.applications import densenet
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam
import datetime

K.clear_session()

base_model = densenet.DenseNet201(include_top=False, weights='imagenet', input_shape=(512, 512, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(len(n_class), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 256, 256, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

# Trainning

In [12]:
train_gen = ImageDataGenerator(horizontal_flip=True, 
                               vertical_flip=True)

X_train, y_train = image_crop(train_idx)
X_val, y_val = image_crop(val_idx)

ValueError: not enough values to unpack (expected 3, got 0)

In [ ]:
# model.load_weights('ieeev2-202-0.10940.hdf5')

model_checkpoint = ModelCheckpoint('ieeev3-{epoch:02d}-{val_loss:.5f}.hdf5',
                                   monitor='val_loss', save_best_only=False, save_weights_only=True)

model_earlystop = EarlyStopping(patience=32, monitor='val_loss')
adlr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=16, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=1e-6)

batch_size = 64

train_history = model.fit_generator(train_gen.flow(X_train, y_train, batch_size), 
                                    epochs=2**8, steps_per_epoch=len(train_idx), 
                                    validation_data=train_gen.flow(X_val, y_val, batch_size), 
                                    validation_steps=len(val_idx),
                                    verbose=2, callbacks=[model_checkpoint, adlr, model_earlystop])

# Testing

In [ ]:
testpath = os.path.join(rootpath, 'test')
submitpath = '/media/share/jiaxin_cmu/kaggle/ieee_camera/submition/'

submit_df = pd.read_csv(rootpath + '/sample_submission.csv')
    
model.load_weights('ieeev2-202-0.10940.hdf5')

prediction = [0]
for k in range(10):
    print(k)
    img_file = []
    for ii, idx in enumerate(submit_df['fname']):
        img = plt.imread(testpath + '/' + idx)

        hr = np.random.randint(0, 256)
        wr = np.random.randint(0, 256)
        tem_img = img[hr:hr+256, wr:wr+256, :]
        img_file.append(tem_img / 255.)
    
    prediction = np.add(model.predict(np.array(img_file)), prediction)


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder().fit(n_class)
predict_class = le.inverse_transform(prediction.argmax(axis=-1))

In [ ]:
submit_df['camera'] = predict_class
submit_df.to_csv(submitpath + '/' + 'submit04.csv', index=False)